In [1]:
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np


import logging

import dvc.api


import patsy as ps
import os, sys
from IPython.display import Image, display

In [2]:
data = pd.read_csv('../data/data_outlier.csv')
data.head(5)

,Unnamed: 0,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,6,M,18.25,19.98,119.6,1040.0,0.09463,0.10900,0.11270,0.07400,...,22.88,27.66,153.2,1606.0,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368
1,7,M,13.71,20.83,90.2,577.9,0.11890,0.16450,0.09366,0.05985,...,17.06,28.14,110.6,897.0,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510
2,10,M,16.02,23.24,102.7,797.8,0.08206,0.06669,0.03299,0.03323,...,19.19,33.88,123.8,1150.0,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452
3,11,M,15.78,17.89,103.6,781.0,0.09710,0.12920,0.09954,0.06606,...,20.42,27.28,136.5,1299.0,0.1396,0.5609,0.3965,0.18100,0.3792,0.10480
4,13,M,15.85,23.95,103.7,782.7,0.08401,0.10020,0.09938,0.05364,...,16.84,27.66,112.0,876.5,0.1131,0.1924,0.2322,0.11190,0.2809,0.06287


In [3]:
data = data.drop('Unnamed: 0',axis = 1)
data

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368
1,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510
2,M,16.02,23.24,102.70,797.8,0.08206,0.06669,0.03299,0.03323,0.1528,...,19.19,33.88,123.80,1150.0,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452
3,M,15.78,17.89,103.60,781.0,0.09710,0.12920,0.09954,0.06606,0.1842,...,20.42,27.28,136.50,1299.0,0.1396,0.5609,0.3965,0.18100,0.3792,0.10480
4,M,15.85,23.95,103.70,782.7,0.08401,0.10020,0.09938,0.05364,0.1847,...,16.84,27.66,112.00,876.5,0.1131,0.1924,0.2322,0.11190,0.2809,0.06287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,B,12.88,28.92,82.50,514.3,0.08123,0.05824,0.06195,0.02343,0.1566,...,13.89,35.74,88.84,595.7,0.1227,0.1620,0.2439,0.06493,0.2372,0.07242
394,B,10.29,27.61,65.67,321.4,0.09030,0.07658,0.05999,0.02738,0.1593,...,10.84,34.91,69.57,357.6,0.1384,0.1710,0.2000,0.09127,0.2226,0.08283
395,B,14.59,22.68,96.39,657.1,0.08473,0.13300,0.10290,0.03736,0.1454,...,15.48,27.27,105.90,733.5,0.1026,0.3171,0.3662,0.11050,0.2258,0.08004
396,B,14.05,27.15,91.38,600.4,0.09929,0.11260,0.04462,0.04304,0.1537,...,15.30,33.17,100.20,706.7,0.1241,0.2264,0.1326,0.10480,0.2250,0.08321


In [4]:

drop_col = ['perimeter_mean','area_mean','perimeter_worst','area_worst', 'area_se','perimeter_se']
data = data.drop(columns=drop_col)
data.head(5)

,diagnosis,radius_mean,texture_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,...,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,18.25,19.98,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,0.4467,...,0.01369,0.002179,22.88,27.66,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368
1,M,13.71,20.83,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,0.5835,...,0.01486,0.005412,17.06,28.14,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510
2,M,16.02,23.24,0.08206,0.06669,0.03299,0.03323,0.1528,0.05697,0.3795,...,0.01460,0.003042,19.19,33.88,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452
3,M,15.78,17.89,0.09710,0.12920,0.09954,0.06606,0.1842,0.06082,0.5058,...,0.02008,0.004144,20.42,27.28,0.1396,0.5609,0.3965,0.18100,0.3792,0.10480
4,M,15.85,23.95,0.08401,0.10020,0.09938,0.05364,0.1847,0.05338,0.4033,...,0.02981,0.003002,16.84,27.66,0.1131,0.1924,0.2322,0.11190,0.2809,0.06287


In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
# creating instance of labelencoder
labelencoder = LabelEncoder()

In [7]:
data['diagnosis'] = labelencoder.fit_transform(data['diagnosis'])
data

,diagnosis,radius_mean,texture_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,...,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,18.25,19.98,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,0.4467,...,0.01369,0.002179,22.88,27.66,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368
1,1,13.71,20.83,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,0.5835,...,0.01486,0.005412,17.06,28.14,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510
2,1,16.02,23.24,0.08206,0.06669,0.03299,0.03323,0.1528,0.05697,0.3795,...,0.01460,0.003042,19.19,33.88,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452
3,1,15.78,17.89,0.09710,0.12920,0.09954,0.06606,0.1842,0.06082,0.5058,...,0.02008,0.004144,20.42,27.28,0.1396,0.5609,0.3965,0.18100,0.3792,0.10480
4,1,15.85,23.95,0.08401,0.10020,0.09938,0.05364,0.1847,0.05338,0.4033,...,0.02981,0.003002,16.84,27.66,0.1131,0.1924,0.2322,0.11190,0.2809,0.06287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,0,12.88,28.92,0.08123,0.05824,0.06195,0.02343,0.1566,0.05708,0.2116,...,0.01695,0.002801,13.89,35.74,0.1227,0.1620,0.2439,0.06493,0.2372,0.07242
394,0,10.29,27.61,0.09030,0.07658,0.05999,0.02738,0.1593,0.06127,0.2199,...,0.01843,0.004938,10.84,34.91,0.1384,0.1710,0.2000,0.09127,0.2226,0.08283
395,0,14.59,22.68,0.08473,0.13300,0.10290,0.03736,0.1454,0.06147,0.2254,...,0.01638,0.004406,15.48,27.27,0.1026,0.3171,0.3662,0.11050,0.2258,0.08004
396,0,14.05,27.15,0.09929,0.11260,0.04462,0.04304,0.1537,0.06171,0.3645,...,0.02080,0.005304,15.30,33.17,0.1241,0.2264,0.1326,0.10480,0.2250,0.08321


In [8]:
from causalnex.structure.notears import from_pandas
sm = from_pandas(data)

In [9]:


from IPython.display import Image
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

In [10]:
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK)
Image(viz.draw(format='png'))

Warning: 
            Pygraphviz not installed. Also make sure you have the system-level
            ``graphviz`` requirement installed.

            Alternatively, you can visualise your graph using the networkx.draw
            functionality:
            >>> sm = StructureModel()
            >>> fig, ax = plt.subplots()
            >>> nx.draw_circular(sm, ax=ax)
            >>> fig.show()
            

In [ ]:
sm.remove_edges_below_threshold(0.8)
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK)
Image(viz.draw(format='png'))

In [ ]:
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK)
Image(viz.draw(format='png'))